In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU detected: {gpus}")
    except RuntimeError as e:
        print(f"Error setting GPU memory growth: {e}")
else:
    print("No GPU detected, training will run on CPU.")

GPU detected: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [7]:
file_path = '../data/medicine_stocks.csv'
df = pd.read_csv(file_path)

In [9]:
df.head()

,id_hospital,id_medicine,input_year,input_month,min_stock,max_stock,current_stock,usage_qty,order_qty,order_date,lead_time,date
0,1,1,2020,1,600,3500,2482,1018,0,2020-01-31,0,2020-01-01
1,1,1,2020,2,600,3500,1330,1152,0,2020-02-29,0,2020-02-01
2,1,1,2020,3,600,3500,3500,1235,3405,2020-03-05,7,2020-03-01
3,1,1,2020,4,600,3500,2192,1308,0,2020-04-30,0,2020-04-01
4,1,1,2020,5,600,3500,1029,1163,0,2020-05-31,0,2020-05-01


In [8]:
df['date'] = pd.to_datetime(df['input_year'].astype(str) + '-' +
                            df['input_month'].astype(str) + '-01')
df.sort_values(['id_medicine', 'date'], inplace=True)

In [10]:
med_id = 1
df_med = df[df['id_medicine'] == med_id].copy()

In [11]:
series = df_med[['date', 'usage_qty']].set_index('date')

In [12]:
scaler = MinMaxScaler(feature_range=(0, 1))
series['usage_scaled'] = scaler.fit_transform(series[['usage_qty']])

In [13]:
def create_sequences(data_array, window_size=12):
    X, y = [], []
    for i in range(len(data_array) - window_size):
        X.append(data_array[i : i + window_size])
        y.append(data_array[i + window_size])
    X = np.array(X)
    y = np.array(y)
    return X.reshape((X.shape[0], X.shape[1], 1)), y.reshape(-1, 1)

In [14]:
values = series['usage_scaled'].values
WINDOW_SIZE = 12
X, y = create_sequences(values, WINDOW_SIZE)

In [15]:
split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

print(f"Total samples: {len(X)}")
print(f"  -> Train samples: {len(X_train)}")
print(f"  -> Test samples:  {len(X_test)}")

Total samples: 48
  -> Train samples: 38
  -> Test samples:  10


In [18]:
model = Sequential([
    Bidirectional(LSTM(50, return_sequences=False), input_shape=(WINDOW_SIZE, 1)),
    Dense(1, activation='linear')
])

model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mean_absolute_error']
)

model.summary()

/home/salsabiila/miniconda3/envs/ml_venv/lib/python3.12/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ (None, 100)            │        20,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,901 (81.64 KB)

 Trainable params: 20,901 (81.64 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
EPOCHS = 50
BATCH_SIZE = 8

In [20]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=2
)

Epoch 1/50


I0000 00:00:1748684276.784540  258762 cuda_dnn.cc:529] Loaded cuDNN version 90501


5/5 - 5s - 913ms/step - loss: 0.2805 - mean_absolute_error: 0.4350 - val_loss: 0.1656 - val_mean_absolute_error: 0.3213
Epoch 2/50
5/5 - 0s - 51ms/step - loss: 0.1398 - mean_absolute_error: 0.3019 - val_loss: 0.0809 - val_mean_absolute_error: 0.2482
Epoch 3/50
5/5 - 0s - 42ms/step - loss: 0.0940 - mean_absolute_error: 0.2565 - val_loss: 0.0752 - val_mean_absolute_error: 0.2502
Epoch 4/50
5/5 - 0s - 47ms/step - loss: 0.1054 - mean_absolute_error: 0.2775 - val_loss: 0.0736 - val_mean_absolute_error: 0.2474
Epoch 5/50
5/5 - 0s - 39ms/step - loss: 0.0936 - mean_absolute_error: 0.2597 - val_loss: 0.0649 - val_mean_absolute_error: 0.2252
Epoch 6/50
5/5 - 0s - 35ms/step - loss: 0.0832 - mean_absolute_error: 0.2475 - val_loss: 0.0644 - val_mean_absolute_error: 0.2249
Epoch 7/50
5/5 - 0s - 37ms/step - loss: 0.0821 - mean_absolute_error: 0.2422 - val_loss: 0.0654 - val_mean_absolute_error: 0.2241
Epoch 8/50
5/5 - 0s - 48ms/step - loss: 0.0803 - mean_absolute_error: 0.2387 - val_loss: 0.0625 - va

In [21]:
loss, mae = model.evaluate(X_test, y_test, verbose=0)
print(f"\n== Test Set Evaluation ==")
print(f"Test Loss (MSE): {loss:.6f}")
print(f"Test MAE       : {mae:.6f}")


== Test Set Evaluation ==
Test Loss (MSE): 0.005107
Test MAE       : 0.052758


In [23]:
model_save_path = 'bilstm_medicine1_model.keras'
model.save(model_save_path)
print(f"\nModel saved to: {model_save_path}")


Model saved to: bilstm_medicine1_model.keras


In [ ]:
hist_df = pd.DataFrame(history.history)
hist_df.to_csv('training_history_bilstm_medicine1.csv', index=False)
print("Training history saved to: training_history_bilstm_medicine1.csv")